# Installations

In [ ]:
!pip install ucimlrepo
!pip install diffprivlib

In [ ]:
!pip install fairlearn

In [ ]:
# Standard Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
# DiffPriv Classifiers
from diffprivlib.models import LogisticRegression as DPLR
from diffprivlib.models import RandomForestClassifier as DPRF
from diffprivlib.models import GaussianNB as DPNB
# Fairness Mitigator
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
# Other
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from ucimlrepo import fetch_ucirepo

In [ ]:
import pandas as pd
import numpy as np

# Fairness Metrices

In [ ]:
def calculate_disparate_impact_ratio(y_true, y_pred, sensitive_features):
    sensitive_groups = sensitive_features.unique()
    favorable_outcome_rates = {}

    for group in sensitive_groups:
        group_indices = sensitive_features[sensitive_features == group].index

        # Ensure y_true and y_pred are pandas Series for consistent indexing
        if not isinstance(y_true, pd.Series):
            y_true = pd.Series(y_true, index=sensitive_features.index)
        if not isinstance(y_pred, pd.Series):
            y_pred = pd.Series(y_pred, index=sensitive_features.index)

        y_true_group = y_true.loc[group_indices]
        y_pred_group = y_pred.loc[group_indices]

        # Calculate the rate of favorable outcomes (predicted as 1) for the group
        # Avoid division by zero if a group is empty
        if len(y_pred_group) > 0:
            favorable_rate = np.sum(y_pred_group == 1) / len(y_pred_group)
        else:
            favorable_rate = 0
        favorable_outcome_rates[group] = favorable_rate

    # Calculate the ratio of favorable outcomes for the unprivileged group to the privileged group
    rates = list(favorable_outcome_rates.values())
    if len(rates) < 2:
        return 1.0  # Disparate impact is 1 if there's only one group

    min_rate = min(rates)
    max_rate = max(rates)

    # Avoid division by zero
    disparate_impact_ratio = min_rate / max_rate if max_rate > 0 else 1.0

    return disparate_impact_ratio

In [ ]:
def calculate_demographic_parity_difference(y_true, y_pred, sensitive_features):
    sensitive_groups = sensitive_features.unique()
    favorable_outcome_rates = {}

    for group in sensitive_groups:
        group_indices = sensitive_features[sensitive_features == group].index

        # Ensure y_true and y_pred are pandas Series for consistent indexing
        if not isinstance(y_true, pd.Series):
            y_true = pd.Series(y_true, index=sensitive_features.index)
        if not isinstance(y_pred, pd.Series):
            y_pred = pd.Series(y_pred, index=sensitive_features.index)

        y_true_group = y_true.loc[group_indices]
        y_pred_group = y_pred.loc[group_indices]

        # Calculate the rate of favorable outcomes (predicted as 1) for the group
        # Avoid division by zero if a group is empty
        if len(y_pred_group) > 0:
            favorable_rate = np.sum(y_pred_group == 1) / len(y_pred_group)
        else:
            favorable_rate = 0
        favorable_outcome_rates[group] = favorable_rate

    # Calculate the difference between the maximum and minimum favorable outcome rates
    max_rate = max(favorable_outcome_rates.values())
    min_rate = min(favorable_outcome_rates.values())
    demographic_parity_difference = max_rate - min_rate

    return demographic_parity_difference

In [ ]:
def calculate_equal_opportunity_difference(y_true, y_pred, sensitive_features):
    sensitive_groups = sensitive_features.unique()
    tpr_by_group = {}

    for group in sensitive_groups:
        group_indices = sensitive_features[sensitive_features == group].index

        # Ensure y_true and y_pred are pandas Series for consistent indexing
        if not isinstance(y_true, pd.Series):
            y_true = pd.Series(y_true, index=sensitive_features.index)
        if not isinstance(y_pred, pd.Series):
            y_pred = pd.Series(y_pred, index=sensitive_features.index)

        y_true_group = y_true.loc[group_indices]
        y_pred_group = y_pred.loc[group_indices]

        # Calculate True Positive Rate (TPR) for the group (Recall)
        true_positives = np.sum((y_true_group == 1) & (y_pred_group == 1))
        actual_positives = np.sum(y_true_group == 1)

        # Avoid division by zero
        tpr = true_positives / actual_positives if actual_positives > 0 else 0

        tpr_by_group[group] = tpr

    # Calculate the difference in TPR between the maximum and minimum across groups
    equal_opportunity_difference = max(tpr_by_group.values()) - min(tpr_by_group.values())

    return equal_opportunity_difference

In [ ]:
def calculate_equalized_odds_difference(y_true, y_pred, sensitive_features):
    sensitive_groups = sensitive_features.unique()
    tpr_by_group = {}
    fpr_by_group = {}

    for group in sensitive_groups:
        group_indices = sensitive_features[sensitive_features == group].index

        # Ensure y_true and y_pred are pandas Series for consistent indexing
        if not isinstance(y_true, pd.Series):
            y_true = pd.Series(y_true, index=sensitive_features.index)
        if not isinstance(y_pred, pd.Series):
            y_pred = pd.Series(y_pred, index=sensitive_features.index)

        y_true_group = y_true.loc[group_indices]
        y_pred_group = y_pred.loc[group_indices]

        # Calculate True Positive Rate (TPR) and False Positive Rate (FPR) for the group
        true_positives = np.sum((y_true_group == 1) & (y_pred_group == 1))
        false_positives = np.sum((y_true_group == 0) & (y_pred_group == 1))
        true_negatives = np.sum((y_true_group == 0) & (y_pred_group == 0))
        false_negatives = np.sum((y_true_group == 1) & (y_pred_group == 0))

        # Avoid division by zero
        tpr = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        fpr = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0

        tpr_by_group[group] = tpr
        fpr_by_group[group] = fpr

    # Calculate the difference in TPR and FPR between the maximum and minimum across groups
    tpr_difference = max(tpr_by_group.values()) - min(tpr_by_group.values())
    fpr_difference = max(fpr_by_group.values()) - min(fpr_by_group.values())

    # Equalized odds difference is the maximum of these two differences
    equalized_odds_difference = max(tpr_difference, fpr_difference)

    return equalized_odds_difference

# Extract Adult Dataset

In [ ]:
title = 'adult'

# Fetch dataset
adult = fetch_ucirepo(id=2)

# Get the data and target (income)
data = adult.data.features
target = adult.data.targets

data = pd.concat([data, target], axis=1)

# Explicitly handle missing values using na_values
data = data.replace('?', pd.NA)

# Drop rows with missing values for CTGAN
data = data.dropna()

data.loc[:, 'income'] = data['income'].str.strip('.')

discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]
print(data.head())
print(data.tail())

num_samples, num_features = data.shape
print(f"Number of samples: {num_samples}")
print(f"Number of features: {num_features}")

   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country income  
0          2174             0              40  United-States  <=50K  
1             0             0             

In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'age':
[39 50 38 53 28 37 49 52 31 42 30 23 32 34 25 43 40 54 35 59 56 19 20 45
 22 48 21 24 57 44 41 29 47 46 36 79 27 18 33 76 55 61 70 64 71 66 51 58
 26 17 60 90 75 65 77 62 63 67 74 72 69 68 73 81 78 88 80 84 83 85 82 86
 89 87]
------------------------------
Unique values for column 'workclass':
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'Self-emp-inc' 'Without-pay']
------------------------------
Unique values for column 'fnlwgt':
[ 77516  83311 215646 ... 173449  89686 350977]
------------------------------
Unique values for column 'education':
['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 '7th-8th' 'Doctorate' 'Assoc-voc' 'Prof-school' '5th-6th' '10th'
 'Preschool' '12th' '1st-4th']
------------------------------
Unique values for column 'education-num':
[13  9  7 14  5 10 12  4 16 11 15  3  6  1  8  2]
------------------------------
Unique values for column 'marital-status':
['Never-married' 'Marr

In [ ]:
# Factorize categorical features
for col in data.select_dtypes(include='object').columns:
    data[col], unique_values = pd.factorize(data[col])

print("Features after factorizing:")
display(data.head())

Features after factorizing:


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1,0


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[['fnlwgt']] = scaler.fit_transform(data[['fnlwgt']])

print("Features after scaling 'fnlwgt'")
display(data.head())

Features after scaling 'fnlwgt'


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,0,-1.062295,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,-1.007438,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,0.245284,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,0.425853,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,1.407393,0,13,1,3,2,1,1,0,0,40,1,0


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'age':
[39 50 38 53 28 37 49 52 31 42 30 23 32 34 25 43 40 54 35 59 56 19 20 45
 22 48 21 24 57 44 41 29 47 46 36 79 27 18 33 76 55 61 70 64 71 66 51 58
 26 17 60 90 75 65 77 62 63 67 74 72 69 68 73 81 78 88 80 84 83 85 82 86
 89 87]
------------------------------
Unique values for column 'workclass':
[0 1 2 3 4 5 6]
------------------------------
Unique values for column 'fnlwgt':
[-1.06229487 -1.00743773  0.24528351 ... -0.15416545 -0.94709014
  1.52636574]
------------------------------
Unique values for column 'education':
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
------------------------------
Unique values for column 'education-num':
[13  9  7 14  5 10 12  4 16 11 15  3  6  1  8  2]
------------------------------
Unique values for column 'marital-status':
[0 1 2 3 4 5 6]
------------------------------
Unique values for column 'occupation':
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
------------------------------
Unique values for column 'relation

In [ ]:
X = data.drop('income', axis=1)
y = data['income']

print("Features (X):")
print(X.head())
print("\nTarget (y):")
print(y.head())

Features (X):
   age  workclass    fnlwgt  education  education-num  marital-status  \
0   39          0 -1.062295          0             13               0   
1   50          1 -1.007438          0             13               1   
2   38          2  0.245284          1              9               2   
3   53          2  0.425853          2              7               1   
4   28          2  1.407393          0             13               1   

   occupation  relationship  race  sex  capital-gain  capital-loss  \
0           0             0     0    0          2174             0   
1           1             1     0    0             0             0   
2           2             0     0    0             0             0   
3           2             1     1    0             0             0   
4           3             2     1    1             0             0   

   hours-per-week  native-country  
0              40               0  
1              13               0  
2              40 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Combine features and labels back together for manipulation
train_df = pd.concat([X_train, y_train], axis=1)

print("Before balancing:")
print(train_df['income'].value_counts())

# Separate into majority (income = 0) and minority (income = 1)
majority_df = train_df[train_df['income'] == 0]
minority_df = train_df[train_df['income'] == 1]

# Randomly drop rows from the majority to match the size of the minority
majority_downsampled = majority_df.sample(n=len(minority_df), random_state=42)

# Combine the minority and downsampled majority
balanced_train_df = pd.concat([majority_downsampled, minority_df])

# Shuffle rows
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate back into features (X) and target (y)
X_train_balanced = balanced_train_df.drop(columns=['income'])
y_train_balanced = balanced_train_df['income']

print("After balancing:")
print(y_train_balanced.value_counts())

Before balancing:
income
0    27269
1     8908
Name: count, dtype: int64
After balancing:
income
1    8908
0    8908
Name: count, dtype: int64


In [ ]:
sensitive_features_train = X_train_balanced['sex']
print("Sensitive features (sex) from X_train_balanced:")
display(sensitive_features_train.head())

Sensitive features (sex) from X_train_balanced:


,sex
0,0
1,0
2,1
3,0
4,0


In [ ]:
sensitive_features = X_test['sex']
indicator = 'binary'

# Extract Compas Dataset

In [ ]:
title = 'compas'

data = pd.read_csv('compas-scores-raw.csv')

In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'Person_ID':
[50844 50848 50855 ... 68595 68598 68603]
------------------------------
Unique values for column 'AssessmentID':
[57167 57174 57181 ... 79660 79665 79669]
------------------------------
Unique values for column 'Case_ID':
[51950 51956 51963 ... 72035 72038 72042]
------------------------------
Unique values for column 'Agency_Text':
['PRETRIAL' 'Probation' 'DRRD' 'Broward County']
------------------------------
Unique values for column 'LastName':
['Fisher' 'KENDALL' 'DAYES' ... 'WEBLEY' 'Sime' 'PENTO']
------------------------------
Unique values for column 'FirstName':
['Kevin' 'KEVIN' 'DANIEL' ... 'STERN' 'krishna' 'Viki']
------------------------------
Unique values for column 'MiddleName':
[nan 'George' 'Nicole' ... 'Wilmington' 'SHEREE' 'THEOPHILUS']
------------------------------
Unique values for column 'Sex_Code_Text':
['Male' 'Female']
------------------------------
Unique values for column 'Ethnic_Code_Text':
['Caucasian' 'African-Ameri

In [ ]:
print(f"Size of data before dropping missing values: {data.shape[0]}")

# Drop rows with missing values
data = data.dropna()

print(f"Size of data after dropping missing values: {data.shape[0]}")

print("Data after dropping rows with missing values and duplicates:")
display(data.head())

Size of data before dropping missing values: 60843
Size of data after dropping missing values: 15615
Data after dropping rows with missing values and duplicates:


,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted
9,50850,57176,51958,PRETRIAL,Debe,Mikerlie,George,Female,African-American,10/09/94,...,2,Medium,7,Risk of Violence,-1.29,7,Medium,New,1,0
10,50850,57176,51958,PRETRIAL,Debe,Mikerlie,George,Female,African-American,10/09/94,...,2,Medium,8,Risk of Recidivism,-0.53,6,Medium,New,1,0
11,50850,57176,51958,PRETRIAL,Debe,Mikerlie,George,Female,African-American,10/09/94,...,2,Medium,18,Risk of Failure to Appear,11.00,1,Low,New,1,0
12,50839,57162,51945,PRETRIAL,McLaurin,Stephanie,Nicole,Female,African-American,06/29/85,...,1,Low,7,Risk of Violence,-2.90,2,Low,New,1,0
13,50839,57162,51945,PRETRIAL,McLaurin,Stephanie,Nicole,Female,African-American,06/29/85,...,1,Low,8,Risk of Recidivism,-1.55,2,Low,New,1,0


In [ ]:
data.drop(columns=['MiddleName', 'LastName', 'FirstName', 'Person_ID', 'AssessmentID', 'Case_ID', 'ScaleSet', 'RecSupervisionLevelText', 'Scale_ID', 'DisplayText', 'RawScore', 'DecileScore', 'AssessmentType', 'AssessmentReason', 'IsCompleted', 'IsDeleted'], axis=0, inplace=True)
display(data.head())

,Agency_Text,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,ScaleSet_ID,Language,LegalStatus,CustodyStatus,MaritalStatus,Screening_Date,RecSupervisionLevel,ScoreText
9,PRETRIAL,Female,African-American,10/09/94,22,English,Pretrial,Jail Inmate,Significant Other,1/1/13 0:00,2,Medium
10,PRETRIAL,Female,African-American,10/09/94,22,English,Pretrial,Jail Inmate,Significant Other,1/1/13 0:00,2,Medium
11,PRETRIAL,Female,African-American,10/09/94,22,English,Pretrial,Jail Inmate,Significant Other,1/1/13 0:00,2,Low
12,PRETRIAL,Female,African-American,06/29/85,22,English,Pretrial,Jail Inmate,Single,1/1/13 0:00,1,Low
13,PRETRIAL,Female,African-American,06/29/85,22,English,Pretrial,Jail Inmate,Single,1/1/13 0:00,1,Low


In [ ]:
data['Screening_Date'] = pd.to_datetime(data['Screening_Date'], format='%m/%d/%y %H:%M')
data['DateOfBirth'] = pd.to_datetime(data['DateOfBirth'], format='%m/%d/%y')
data['Age'] = (data['Screening_Date'] - data['DateOfBirth']).dt.days // 365
data.drop(columns=['DateOfBirth', 'Screening_Date'], inplace=True)

display(data.head())

,Agency_Text,Sex_Code_Text,Ethnic_Code_Text,ScaleSet_ID,Language,LegalStatus,CustodyStatus,MaritalStatus,RecSupervisionLevel,ScoreText,Age
9,PRETRIAL,Female,African-American,22,English,Pretrial,Jail Inmate,Significant Other,2,Medium,18
10,PRETRIAL,Female,African-American,22,English,Pretrial,Jail Inmate,Significant Other,2,Medium,18
11,PRETRIAL,Female,African-American,22,English,Pretrial,Jail Inmate,Significant Other,2,Low,18
12,PRETRIAL,Female,African-American,22,English,Pretrial,Jail Inmate,Single,1,Low,27
13,PRETRIAL,Female,African-American,22,English,Pretrial,Jail Inmate,Single,1,Low,27


In [ ]:
data['ScoreText'] = data['ScoreText'].apply(lambda x: 1 if x == 'High' else 0)

print("ScoreText after binarization:")
print(data['ScoreText'].value_counts())

ScoreText after binarization:
ScoreText
0    13641
1     1974
Name: count, dtype: int64


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'Agency_Text':
['PRETRIAL' 'Probation' 'DRRD' 'Broward County']
------------------------------
Unique values for column 'Sex_Code_Text':
['Female' 'Male']
------------------------------
Unique values for column 'Ethnic_Code_Text':
['African-American' 'Other' 'Caucasian' 'Asian' 'Hispanic'
 'Native American' 'African-Am' 'Oriental' 'Arabic']
------------------------------
Unique values for column 'ScaleSet_ID':
[22 17]
------------------------------
Unique values for column 'Language':
['English' 'Spanish']
------------------------------
Unique values for column 'LegalStatus':
['Pretrial' 'Post Sentence' 'Other' 'Probation Violator'
 'Conditional Release']
------------------------------
Unique values for column 'CustodyStatus':
['Jail Inmate' 'Probation' 'Pretrial Defendant' 'Parole' 'Prison Inmate'
 'Residential Program']
------------------------------
Unique values for column 'MaritalStatus':
['Significant Other' 'Single' 'Married' 'Divorced' 'Separated' 'Wido

In [ ]:
# Factorize categorical features
for col in data.select_dtypes(include='object').columns:
    data[col], unique_values = pd.factorize(data[col])

print("Features after factorizing:")
display(data.head())

Features after factorizing:


,Agency_Text,Sex_Code_Text,Ethnic_Code_Text,ScaleSet_ID,Language,LegalStatus,CustodyStatus,MaritalStatus,RecSupervisionLevel,ScoreText,Age
9,0,0,0,22,0,0,0,0,2,0,18
10,0,0,0,22,0,0,0,0,2,0,18
11,0,0,0,22,0,0,0,0,2,0,18
12,0,0,0,22,0,0,0,1,1,0,27
13,0,0,0,22,0,0,0,1,1,0,27


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'Agency_Text':
[0 1 2 3]
------------------------------
Unique values for column 'Sex_Code_Text':
[0 1]
------------------------------
Unique values for column 'Ethnic_Code_Text':
[0 1 2 3 4 5 6 7 8]
------------------------------
Unique values for column 'ScaleSet_ID':
[22 17]
------------------------------
Unique values for column 'Language':
[0 1]
------------------------------
Unique values for column 'LegalStatus':
[0 1 2 3 4]
------------------------------
Unique values for column 'CustodyStatus':
[0 1 2 3 4 5]
------------------------------
Unique values for column 'MaritalStatus':
[0 1 2 3 4 5 6]
------------------------------
Unique values for column 'RecSupervisionLevel':
[2 1 3 4]
------------------------------
Unique values for column 'ScoreText':
[0 1]
------------------------------
Unique values for column 'Age':
[ 18  27  40  26  19  33  34  42  23 -48  25  21 -31  35  29  20  41  28
  22  30 -55  24 -32 -52 -44  43  37  31 -54 -49  38 -38  32 -4

In [ ]:
X = data.drop('ScoreText', axis=1)
y = data['ScoreText']

print("Features (X):")
print(X.head())
print("\nTarget (y):")
print(y.head())

Features (X):
    Agency_Text  Sex_Code_Text  Ethnic_Code_Text  ScaleSet_ID  Language  \
9             0              0                 0           22         0   
10            0              0                 0           22         0   
11            0              0                 0           22         0   
12            0              0                 0           22         0   
13            0              0                 0           22         0   

    LegalStatus  CustodyStatus  MaritalStatus  RecSupervisionLevel  Age  
9             0              0              0                    2   18  
10            0              0              0                    2   18  
11            0              0              0                    2   18  
12            0              0              1                    1   27  
13            0              0              1                    1   27  

Target (y):
9     0
10    0
11    0
12    0
13    0
Name: ScoreText, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Combine features and labels back together for manipulation
train_df = pd.concat([X_train, y_train], axis=1)

print("Before balancing:")
print(train_df['ScoreText'].value_counts())

Before balancing:
ScoreText
0    10929
1     1563
Name: count, dtype: int64


In [ ]:
# Combine features and labels back together for manipulation
train_df = pd.concat([X_train, y_train], axis=1)

print("Before balancing:")
print(train_df['ScoreText'].value_counts())

# Identify the minority class and its size
minority_class = train_df['ScoreText'].value_counts().idxmin()
minority_size = train_df['ScoreText'].value_counts().min()

# Separate data by class
class_dfs = {label: train_df[train_df['ScoreText'] == label] for label in train_df['ScoreText'].unique()}

# Downsample majority classes and keep the minority class as is
balanced_class_dfs = []
for label, df in class_dfs.items():
    if label == minority_class:
        balanced_class_dfs.append(df)
    else:
        balanced_class_dfs.append(df.sample(n=minority_size, random_state=42))

# Combine the balanced class dataframes
balanced_train_df = pd.concat(balanced_class_dfs)

# Shuffle rows
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate back into features (X) and target (y)
X_train_balanced = balanced_train_df.drop(columns=['ScoreText'])
y_train_balanced = balanced_train_df['ScoreText']

print("After balancing:")
print(y_train_balanced.value_counts())

Before balancing:
ScoreText
0    10929
1     1563
Name: count, dtype: int64
After balancing:
ScoreText
0    1563
1    1563
Name: count, dtype: int64


In [ ]:
sensitive_features_train = X_train_balanced['Sex_Code_Text']
print("Sensitive features (sex) from X_train_balanced:")
display(sensitive_features_train.head())

Sensitive features (sex) from X_train_balanced:


,Sex_Code_Text
0,1
1,1
2,1
3,1
4,1


In [ ]:
sensitive_features = X_test['Sex_Code_Text']
indicator = 'binary'

# Extract Student Performance Dataset

In [ ]:
# Define the title for the CSV file
title = 'student_performance'

# fetch dataset
student_performance = fetch_ucirepo(id=320)

# data (as pandas dataframes)
X = student_performance.data.features
y = student_performance.data.targets

# metadata
print(student_performance.metadata)

# variable information
print(student_performance.variables)

data = pd.concat([X, y], axis=1)

{'uci_id': 320, 'name': 'Student Performance', 'repository_url': 'https://archive.ics.uci.edu/dataset/320/student+performance', 'data_url': 'https://archive.ics.uci.edu/static/public/320/data.csv', 'abstract': 'Predict student performance in secondary education (high school). ', 'area': 'Social Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 649, 'num_features': 30, 'feature_types': ['Integer'], 'demographics': ['Sex', 'Age', 'Other', 'Education Level', 'Occupation'], 'target_col': ['G1', 'G2', 'G3'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5TG7T', 'creators': ['Paulo Cortez'], 'intro_paper': {'ID': 360, 'type': 'NATIVE', 'title': 'Using data mining to predict secondary school student performance', 'authors': 'P. Cortez, A. M. G. Silva', 'venue': 'Proceedings of 5th Annual Future Business Technolo

In [ ]:
print(f"Size of data before dropping missing values: {data.shape[0]}")

# Drop rows with missing values
data = data.dropna()

print(f"Size of data after dropping missing values: {data.shape[0]}")

print("Data after dropping rows with missing values and duplicates:")
display(data.head())

Size of data before dropping missing values: 649
Size of data after dropping missing values: 649
Data after dropping rows with missing values and duplicates:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'school':
['GP' 'MS']
------------------------------
Unique values for column 'sex':
['F' 'M']
------------------------------
Unique values for column 'age':
[18 17 15 16 19 22 20 21]
------------------------------
Unique values for column 'address':
['U' 'R']
------------------------------
Unique values for column 'famsize':
['GT3' 'LE3']
------------------------------
Unique values for column 'Pstatus':
['A' 'T']
------------------------------
Unique values for column 'Medu':
[4 1 3 2 0]
------------------------------
Unique values for column 'Fedu':
[4 1 2 3 0]
------------------------------
Unique values for column 'Mjob':
['at_home' 'health' 'other' 'services' 'teacher']
------------------------------
Unique values for column 'Fjob':
['teacher' 'other' 'services' 'health' 'at_home']
------------------------------
Unique values for column 'reason':
['course' 'other' 'home' 'reputation']
------------------------------
Unique values for column 'guardian':
['m

In [ ]:
# Define the columns to be mapped from 'yes'/'no' to 1/0
binary_cols = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

# Map 'yes' to 1 and 'no' to 0 for the specified binary columns
for col in binary_cols:
    data[col] = data[col].map({'yes': 1, 'no': 0})

# Factorize the remaining categorical features (excluding the binary columns already mapped)
categorical_cols = data.select_dtypes(include='object').columns
for col in categorical_cols:
    data[col], unique_values = pd.factorize(data[col])

print("Features after preprocessing:")
display(data.head())

Features after preprocessing:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,18,0,0,0,4,4,0,0,...,4,3,4,1,1,3,4,0,11,11
1,0,0,17,0,0,1,1,1,0,1,...,5,3,3,1,1,3,2,9,11,11
2,0,0,15,0,1,1,1,1,0,1,...,4,3,2,2,3,3,6,12,13,12
3,0,0,15,0,0,1,4,2,1,2,...,3,2,2,1,1,5,0,14,14,14
4,0,0,16,0,0,1,3,3,2,1,...,4,3,2,1,2,5,0,11,13,13


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'school':
[0 1]
------------------------------
Unique values for column 'sex':
[0 1]
------------------------------
Unique values for column 'age':
[18 17 15 16 19 22 20 21]
------------------------------
Unique values for column 'address':
[0 1]
------------------------------
Unique values for column 'famsize':
[0 1]
------------------------------
Unique values for column 'Pstatus':
[0 1]
------------------------------
Unique values for column 'Medu':
[4 1 3 2 0]
------------------------------
Unique values for column 'Fedu':
[4 1 2 3 0]
------------------------------
Unique values for column 'Mjob':
[0 1 2 3 4]
------------------------------
Unique values for column 'Fjob':
[0 1 2 3 4]
------------------------------
Unique values for column 'reason':
[0 1 2 3]
------------------------------
Unique values for column 'guardian':
[0 1 2]
------------------------------
Unique values for column 'traveltime':
[2 1 3 4]
------------------------------
Unique values f

In [ ]:
final = data['G3'].apply(lambda x: 1 if x > 12 else 0)

print("G3 after binarization:")
print(final.value_counts())

G3 after binarization:
G3
0    373
1    276
Name: count, dtype: int64


In [ ]:
data['G3'] = final

print("G3 after binarization:")
print(data['G3'].value_counts())

G3 after binarization:
G3
0    373
1    276
Name: count, dtype: int64


In [ ]:
X = data.drop('G3', axis=1)
y = data['G3']

print("Features (X):")
display(X.head())
print("\nTarget (y):")
display(y.head())

Features (X):


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,0,0,18,0,0,0,4,4,0,0,...,0,4,3,4,1,1,3,4,0,11
1,0,0,17,0,0,1,1,1,0,1,...,0,5,3,3,1,1,3,2,9,11
2,0,0,15,0,1,1,1,1,0,1,...,0,4,3,2,2,3,3,6,12,13
3,0,0,15,0,0,1,4,2,1,2,...,1,3,2,2,1,1,5,0,14,14
4,0,0,16,0,0,1,3,3,2,1,...,0,4,3,2,1,2,5,0,11,13



Target (y):


,G3
0,0
1,0
2,0
3,1
4,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Combine features and labels back together for manipulation
train_df = pd.concat([X_train, y_train], axis=1)

print("Before balancing:")
print(train_df['G3'].value_counts())

# Identify the minority class and its size
minority_class = train_df['G3'].value_counts().idxmin()
minority_size = train_df['G3'].value_counts().min()

# Separate data by class
class_dfs = {label: train_df[train_df['G3'] == label] for label in train_df['G3'].unique()}

# Downsample majority classes and keep the minority class as is
balanced_class_dfs = []
for label, df in class_dfs.items():
    if label == minority_class:
        balanced_class_dfs.append(df)
    else:
        balanced_class_dfs.append(df.sample(n=minority_size, random_state=42))

# Combine the balanced class dataframes
balanced_train_df = pd.concat(balanced_class_dfs)

# Shuffle rows
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate back into features (X) and target (y)
X_train_balanced = balanced_train_df.drop(columns=['G3'])
y_train_balanced = balanced_train_df['G3']

print("After balancing:")
print(y_train_balanced.value_counts())

Before balancing:
G3
0    300
1    219
Name: count, dtype: int64
After balancing:
G3
0    219
1    219
Name: count, dtype: int64


In [ ]:
sensitive_features_train = X_train_balanced['sex']
print("Sensitive features (sex) from X_train_balanced:")
display(sensitive_features_train.head())

Sensitive features (sex) from X_train_balanced:


,sex
0,0
1,0
2,1
3,1
4,0


In [ ]:
sensitive_features = X_test['sex']
indicator = 'binary'

# Extract Diabetes Dataset

In [ ]:
title = 'diabetes'

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

# metadata
print(cdc_diabetes_health_indicators.metadata)

# variable information
print(cdc_diabetes_health_indicators.variables)

data = pd.concat([X, y], axis=1)

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

In [ ]:
print(f"Size of data before dropping missing values: {data.shape[0]}")

# Drop rows with missing values
data = data.dropna()

print(f"Size of data after dropping missing values: {data.shape[0]}")

print("Data after dropping rows with missing values and duplicates:")
display(data.head())

Size of data before dropping missing values: 253680
Size of data after dropping missing values: 253680
Data after dropping rows with missing values and duplicates:


,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'HighBP':
[1 0]
------------------------------
Unique values for column 'HighChol':
[1 0]
------------------------------
Unique values for column 'CholCheck':
[1 0]
------------------------------
Unique values for column 'BMI':
[40 25 28 27 24 30 34 26 33 21 23 22 38 32 37 31 29 20 35 45 39 19 47 18
 36 43 55 49 42 17 16 41 44 50 59 48 52 46 54 57 53 14 15 51 58 63 61 56
 74 62 64 66 73 85 60 67 65 70 82 79 92 68 72 88 96 13 81 71 75 12 77 69
 76 87 89 84 95 98 91 86 83 80 90 78]
------------------------------
Unique values for column 'Smoker':
[1 0]
------------------------------
Unique values for column 'Stroke':
[0 1]
------------------------------
Unique values for column 'HeartDiseaseorAttack':
[0 1]
------------------------------
Unique values for column 'PhysActivity':
[0 1]
------------------------------
Unique values for column 'Fruits':
[0 1]
------------------------------
Unique values for column 'Veggies':
[1 0]
------------------------------
Unique

In [ ]:
X = data.drop('Diabetes_binary', axis=1)
y = data['Diabetes_binary']

print("Features (X):")
display(X.head())
print("\nTarget (y):")
display(y.head())

Features (X):


,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,1,1,40,1,0,0,0,0,1,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,25,1,0,0,1,0,0,...,0,1,3,0,0,0,0,7,6,1
2,1,1,1,28,0,0,0,0,1,0,...,1,1,5,30,30,1,0,9,4,8
3,1,0,1,27,0,0,0,1,1,1,...,1,0,2,0,0,0,0,11,3,6
4,1,1,1,24,0,0,0,1,1,1,...,1,0,2,3,0,0,0,11,5,4



Target (y):


,Diabetes_binary
0,0
1,0
2,0
3,0
4,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Combine features and labels back together for manipulation
train_df = pd.concat([X_train, y_train], axis=1)

print("Before balancing:")
print(train_df['Diabetes_binary'].value_counts())

# Identify the minority class and its size
minority_class = train_df['Diabetes_binary'].value_counts().idxmin()
minority_size = train_df['Diabetes_binary'].value_counts().min()

# Separate data by class
class_dfs = {label: train_df[train_df['Diabetes_binary'] == label] for label in train_df['Diabetes_binary'].unique()}

# Downsample majority classes and keep the minority class as is
balanced_class_dfs = []
for label, df in class_dfs.items():
    if label == minority_class:
        balanced_class_dfs.append(df)
    else:
        balanced_class_dfs.append(df.sample(n=minority_size, random_state=42))

# Combine the balanced class dataframes
balanced_train_df = pd.concat(balanced_class_dfs)

# Shuffle rows
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate back into features (X) and target (y)
X_train_balanced = balanced_train_df.drop(columns=['Diabetes_binary'])
y_train_balanced = balanced_train_df['Diabetes_binary']

print("After balancing:")
print(y_train_balanced.value_counts())

Before balancing:
Diabetes_binary
0    174595
1     28349
Name: count, dtype: int64
After balancing:
Diabetes_binary
1    28349
0    28349
Name: count, dtype: int64


In [ ]:
sensitive_features_train = X_train_balanced['Sex']
print("Sensitive features (sex) from X_train_balanced:")
display(sensitive_features_train.head())

Sensitive features (sex) from X_train_balanced:


,Sex
0,0
1,1
2,0
3,0
4,1


In [ ]:
sensitive_features = X_test['Sex']
indicator = 'binary'

# Extract AIDs Dataset

In [ ]:
title = 'aids'

In [ ]:
# fetch dataset
aids_clinical_trials_group_study_175 = fetch_ucirepo(id=890)

# data (as pandas dataframes)
X = aids_clinical_trials_group_study_175.data.features
y = aids_clinical_trials_group_study_175.data.targets

# metadata
print(aids_clinical_trials_group_study_175.metadata)

# variable information
print(aids_clinical_trials_group_study_175.variables)

data = pd.concat([X, y], axis=1)

{'uci_id': 890, 'name': 'AIDS Clinical Trials Group Study 175', 'repository_url': 'https://archive.ics.uci.edu/dataset/890/aids+clinical+trials+group+study+175', 'data_url': 'https://archive.ics.uci.edu/static/public/890/data.csv', 'abstract': 'The AIDS Clinical Trials Group Study 175 Dataset contains healthcare statistics and categorical information about patients who have been diagnosed with AIDS. This dataset was initially published in 1996. The prediction task is to predict whether or not each patient died within a certain window of time or not. ', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 2139, 'num_features': 23, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Sexual Orientation', 'Race', 'Gender'], 'target_col': ['cid'], 'index_col': ['pidnum'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1996, 'last_updated': 'Fri Nov 

In [ ]:
print(f"Size of data before dropping missing values: {data.shape[0]}")

# Drop rows with missing values
data = data.dropna()

print(f"Size of data after dropping missing values: {data.shape[0]}")

print("Data after dropping rows with missing values and duplicates:")
display(data.head())

Size of data before dropping missing values: 2139
Size of data after dropping missing values: 2139
Data after dropping rows with missing values and duplicates:


,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,cid
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [ ]:
for col in data.columns:
    print(f"Unique values for column '{col}':")
    print(data[col].unique())
    print("-" * 30)

Unique values for column 'time':
[ 948 1002  961 1166 1090 1181  794  957  198  188 1073 1175 1203 1065
  357  308 1062 1069  322 1119 1154 1231  588 1182  644  486 1066 1192
  227  318 1093  434 1206  477 1074  779 1159  813 1087 1014 1020 1139
 1145 1129  391  232  987 1096  962 1089 1068 1056  998  996  995  977
  727  975  958  367  513  575  950  802  811  853 1223 1214 1064  812
  771 1230 1127 1083 1134 1138 1195 1185 1085  330  217 1137 1072  820
 1122 1104 1039  561  877 1180 1209  650 1101  684  973 1187 1164 1160
 1044  241 1112  729 1082 1032 1049  551  999 1117 1211  460  882 1071
 1105 1110  980 1146 1133  739 1047  959  974 1028 1057 1023  904 1157
 1077  994 1153 1081  643  981  563 1140  628  731 1132  483  668 1116
  880 1109 1016 1091  930 1059  947  767 1088  238  983  159  982  230
  137  885  863   55  735 1034 1027  581 1006  778  321  985 1098  910
  941 1094  766  392 1012  736  243 1126 1001 1013  626 1202 1167  817
 1115 1125  986 1152  720  567 1176 1121  89

In [ ]:
# Drop the 'zprior' column
data.drop(columns=['zprior'], inplace=True)

print("Data after dropping 'zprior' column:")
display(data.head())

Data after dropping 'zprior' column:


,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,cid
0,948,2,48,89.8128,0,0,0,100,0,0,...,0,1,0,1,0,422,477,566,324,0
1,1002,3,61,49.4424,0,0,0,90,0,1,...,1,3,0,1,0,162,218,392,564,1
2,961,3,45,88.4520,0,1,1,90,0,1,...,1,3,0,1,1,326,274,2063,1893,0
3,1166,3,47,85.2768,0,1,0,100,0,1,...,1,3,0,1,0,287,394,1590,966,0
4,1090,0,43,66.6792,0,1,0,100,0,1,...,1,3,0,0,0,504,353,870,782,0


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[['time', 'wtkg', 'preanti', 'cd40', 'cd420', 'cd80', 'cd820']] = scaler.fit_transform(data[['time', 'wtkg', 'preanti', 'cd40', 'cd420', 'cd80', 'cd820']])

print("Features after scaling:")
display(data.head())

Features after scaling:


,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820,cid
0,0.235799,2,48,1.107649,0,0,0,100,0,0,...,0,1,0,1,0,0.603131,0.730927,-0.876151,-1.374260,0
1,0.420600,3,61,-1.936862,0,0,0,90,0,1,...,1,3,0,1,0,-1.590108,-1.060207,-1.238586,-0.834779,1
2,0.280288,3,45,1.005025,0,1,1,90,0,1,...,1,3,0,1,1,-0.206680,-0.672935,2.242044,2.152597,0
3,0.981848,3,47,0.765569,0,1,0,100,0,1,...,1,3,0,1,0,-0.535666,0.156934,1.256802,0.068852,0
4,0.721757,0,43,-0.636959,0,1,0,100,0,1,...,1,3,0,0,0,1.294845,-0.126605,-0.242930,-0.344750,0


In [ ]:
X = data.drop('cid', axis=1)
y = data['cid']

print("Features (X):")
display(X.head())
print("\nTarget (y):")
display(y.head())

Features (X):


,time,trt,age,wtkg,hemo,homo,drugs,karnof,oprior,z30,...,gender,str2,strat,symptom,treat,offtrt,cd40,cd420,cd80,cd820
0,0.235799,2,48,1.107649,0,0,0,100,0,0,...,0,0,1,0,1,0,0.603131,0.730927,-0.876151,-1.374260
1,0.420600,3,61,-1.936862,0,0,0,90,0,1,...,0,1,3,0,1,0,-1.590108,-1.060207,-1.238586,-0.834779
2,0.280288,3,45,1.005025,0,1,1,90,0,1,...,1,1,3,0,1,1,-0.206680,-0.672935,2.242044,2.152597
3,0.981848,3,47,0.765569,0,1,0,100,0,1,...,1,1,3,0,1,0,-0.535666,0.156934,1.256802,0.068852
4,0.721757,0,43,-0.636959,0,1,0,100,0,1,...,1,1,3,0,0,0,1.294845,-0.126605,-0.242930,-0.344750



Target (y):


,cid
0,0
1,1
2,0
3,0
4,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Combine features and labels back together for manipulation
train_df = pd.concat([X_train, y_train], axis=1)

print("Before balancing:")
print(train_df['cid'].value_counts())

# Identify the minority class and its size
minority_class = train_df['cid'].value_counts().idxmin()
minority_size = train_df['cid'].value_counts().min()

# Separate data by class
class_dfs = {label: train_df[train_df['cid'] == label] for label in train_df['cid'].unique()}

# Downsample majority classes and keep the minority class as is
balanced_class_dfs = []
for label, df in class_dfs.items():
    if label == minority_class:
        balanced_class_dfs.append(df)
    else:
        balanced_class_dfs.append(df.sample(n=minority_size, random_state=42))

# Combine the balanced class dataframes
balanced_train_df = pd.concat(balanced_class_dfs)

# Shuffle rows
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate back into features (X) and target (y)
X_train_balanced = balanced_train_df.drop(columns=['cid'])
y_train_balanced = balanced_train_df['cid']

print("After balancing:")
print(y_train_balanced.value_counts())

Before balancing:
cid
0    1291
1     420
Name: count, dtype: int64
After balancing:
cid
1    420
0    420
Name: count, dtype: int64


In [ ]:
sensitive_features_train = X_train_balanced['gender']
print("Sensitive features (sex) from X_train_balanced:")
display(sensitive_features_train.head())

Sensitive features (sex) from X_train_balanced:


,gender
0,1
1,1
2,1
3,1
4,1


In [ ]:
sensitive_features = X_test['gender']
indicator = 'binary'

# Standard

In [ ]:
lr_model = LogisticRegression(random_state=42, max_iter=5000)
lr_model.fit(X_train_balanced, y_train_balanced)

y_pred_lr = lr_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_lr)
precision = precision_score(y_test, y_pred_lr, average=indicator)
recall = recall_score(y_test, y_pred_lr, average=indicator)
f1 = f1_score(y_test, y_pred_lr, average=indicator)

dir = calculate_disparate_impact_ratio(y_test, y_pred_lr, sensitive_features)
dpd = calculate_demographic_parity_difference(y_test, y_pred_lr, sensitive_features)
eopp = calculate_equal_opportunity_difference(y_test, y_pred_lr, sensitive_features)
eo = calculate_equalized_odds_difference(y_test, y_pred_lr, sensitive_features)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Disparate Impact Ratio: {dir:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equal Opportunity Difference: {eopp:.4f}")
print(f"Equalized Odds Difference: {eo:.4f}")

lr_result = pd.DataFrame({
    'Model': ['Logistic Regression'],
    'Type': ['Standard'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-score': [f1],
    'DIR': [dir],
    'DPD': [dpd],
    'EOpp': [eopp],
    'EO': [eo]
})

Accuracy: 0.8505
Precision: 0.6529
Recall: 0.7822
F1-score: 0.7117
Disparate Impact Ratio: 0.8938
Demographic Parity Difference: 0.0306
Equal Opportunity Difference: 0.1125
Equalized Odds Difference: 0.1125


In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_balanced, y_train_balanced)

y_pred_rf = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf, average=indicator)
recall = recall_score(y_test, y_pred_rf, average=indicator)
f1 = f1_score(y_test, y_pred_rf, average=indicator)

dir = calculate_disparate_impact_ratio(y_test, y_pred_rf, sensitive_features)
dpd = calculate_demographic_parity_difference(y_test, y_pred_rf, sensitive_features)
eopp = calculate_equal_opportunity_difference(y_test, y_pred_rf, sensitive_features)
eo = calculate_equalized_odds_difference(y_test, y_pred_rf, sensitive_features)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Disparate Impact Ratio: {dir:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equal Opportunity Difference: {eopp:.4f}")
print(f"Equalized Odds Difference: {eo:.4f}")

rf_result = pd.DataFrame({
    'Model': ['Random Forest'],
    'Type': ['Standard'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-score': [f1],
    'DIR': [dir],
    'DPD': [dpd],
    'EO': [eo],
    'EOpp': [eopp]
})

Accuracy: 0.8645
Precision: 0.6617
Recall: 0.8713
F1-score: 0.7521
Disparate Impact Ratio: 0.9589
Demographic Parity Difference: 0.0128
Equal Opportunity Difference: 0.1441
Equalized Odds Difference: 0.1441


In [ ]:
nb_model = GaussianNB()
nb_model.fit(X_train_balanced, y_train_balanced)

y_pred_nb = nb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_nb)
precision = precision_score(y_test, y_pred_nb, average=indicator)
recall = recall_score(y_test, y_pred_nb, average=indicator)
f1 = f1_score(y_test, y_pred_nb, average=indicator)

dir = calculate_disparate_impact_ratio(y_test, y_pred_nb, sensitive_features)
dpd = calculate_demographic_parity_difference(y_test, y_pred_nb, sensitive_features)
eopp = calculate_equal_opportunity_difference(y_test, y_pred_nb, sensitive_features)
eo = calculate_equalized_odds_difference(y_test, y_pred_nb, sensitive_features)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Disparate Impact Ratio: {dir:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equal Opportunity Difference: {eopp:.4f}")
print(f"Equalized Odds Difference: {eo:.4f}")

nb_result = pd.DataFrame({
    'Model': ['Naive Bayes'],
    'Type': ['Standard'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-score': [f1],
    'DIR': [dir],
    'DPD': [dpd],
    'EOpp': [eopp],
    'EO': [eo]
})

Accuracy: 0.7593
Precision: 0.4931
Recall: 0.7030
F1-score: 0.5796
Disparate Impact Ratio: 0.6846
Demographic Parity Difference: 0.1119
Equal Opportunity Difference: 0.1669
Equalized Odds Difference: 0.1669


# DiffPriv

In [ ]:
diffpriv_lr_result = {}
diffpriv_rf_result = {}
diffpriv_nb_result = {}

In [ ]:
epsilon_values = [0.1, 0.5, 1, 5, 10]

for epsilon in epsilon_values:
    print(f"Training with epsilon = {epsilon}")
    dp_lr_model = DPLR(epsilon=epsilon, random_state=42, max_iter=5000)
    dp_lr_model.fit(X_train_balanced, y_train_balanced)

    y_pred_dp_lr = dp_lr_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_dp_lr)
    precision = precision_score(y_test, y_pred_dp_lr, average=indicator)
    recall = recall_score(y_test, y_pred_dp_lr, average=indicator)
    f1 = f1_score(y_test, y_pred_dp_lr, average=indicator)
    dir = calculate_disparate_impact_ratio(y_test, y_pred_dp_lr, sensitive_features)
    dpd = calculate_demographic_parity_difference(y_test, y_pred_dp_lr, sensitive_features)
    eopp = calculate_equal_opportunity_difference(y_test, y_pred_dp_lr, sensitive_features)
    eo = calculate_equalized_odds_difference(y_test, y_pred_dp_lr, sensitive_features)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Disparate Impact Ratio: {dir:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    print(f"Equal Opportunity Difference: {eopp:.4f}")
    print(f"Equalized Odds Difference: {eo:.4f}")

    # Count unique values in the prediction
    unique, counts = np.unique(y_pred_dp_lr, return_counts=True)
    print(f"Unique values in prediction: {unique}")
    print(f"Counts of unique values: {counts}")

    del dp_lr_model
    del y_pred_dp_lr

    diffpriv_lr_result[epsilon] = {
        'Model': ['Logistic Regression'],
        'Type': ['DiffPriv'],
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'DIR': dir,
        'DPD': dpd,
        'EOpp': eopp,
        'EO': eo,
    }

    print("-" * 30)

Training with epsilon = 0.1
Accuracy: 0.5794
Precision: 0.2932
Recall: 0.5545
F1-score: 0.3836
Disparate Impact Ratio: 0.6658
Demographic Parity Difference: 0.1578
Equal Opportunity Difference: 0.4360
Equalized Odds Difference: 0.4360
Unique values in prediction: [0 1]
Counts of unique values: [237 191]
------------------------------
Training with epsilon = 0.5
Accuracy: 0.5888
Precision: 0.3057
Recall: 0.5842
F1-score: 0.4014
Disparate Impact Ratio: 0.6580
Demographic Parity Difference: 0.1634
Equal Opportunity Difference: 0.3971
Equalized Odds Difference: 0.3971
Unique values in prediction: [0 1]
Counts of unique values: [235 193]
------------------------------
Training with epsilon = 1
Accuracy: 0.6121
Precision: 0.3224
Recall: 0.5842
F1-score: 0.4155
Disparate Impact Ratio: 0.6275
Demographic Parity Difference: 0.1696
Equal Opportunity Difference: 0.3971
Equalized Odds Difference: 0.3971
Unique values in prediction: [0 1]
Counts of unique values: [245 183]
-------------------------

/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning:

In [ ]:
epsilon_values = [0.1, 0.5, 1, 5, 10]

# Calculate bounds for each feature from the training data
min_bounds = X_train_balanced.min(axis=0).tolist()
max_bounds = X_train_balanced.max(axis=0).tolist()
bounds = (min_bounds, max_bounds)

# Get the unique classes from the training target variable
classes = np.unique(y_train_balanced)

for epsilon in epsilon_values:
    print(f"Training with epsilon = {epsilon}")
    # Pass the calculated bounds and classes to the DPRF constructor
    dp_rf_model = DPRF(epsilon=epsilon, random_state=42, bounds=bounds, classes=classes)
    dp_rf_model.fit(X_train_balanced, y_train_balanced)

    y_pred_dp_rf = dp_rf_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_dp_rf)
    precision = precision_score(y_test, y_pred_dp_rf, average=indicator)
    recall = recall_score(y_test, y_pred_dp_rf, average=indicator)
    f1 = f1_score(y_test, y_pred_dp_rf, average=indicator)
    dir = calculate_disparate_impact_ratio(y_test, y_pred_dp_rf, sensitive_features)
    dpd = calculate_demographic_parity_difference(y_test, y_pred_dp_rf, sensitive_features)
    eopp = calculate_equal_opportunity_difference(y_test, y_pred_dp_rf, sensitive_features)
    eo = calculate_equalized_odds_difference(y_test, y_pred_dp_rf, sensitive_features)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Disparate Impact Ratio: {dir:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    print(f"Equal Opportunity Difference: {eopp:.4f}")
    print(f"Equalized Odds Difference: {eo:.4f}")

    # Count unique values in the prediction
    unique, counts = np.unique(y_pred_dp_rf, return_counts=True)
    print(f"Unique values in prediction: {unique}")
    print(f"Counts of unique values: {counts}")

    del dp_rf_model
    del y_pred_dp_rf

    diffpriv_rf_result[epsilon] = {
        'Model': ['Random Forest'],
        'Type': ['DiffPriv'],
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'DIR': dir,
        'DPD': dpd,
        'EOpp': eopp,
        'EO': eo,
    }

    print("-" * 30)

Training with epsilon = 0.1
Accuracy: 0.5561
Precision: 0.2891
Recall: 0.6040
F1-score: 0.3910
Disparate Impact Ratio: 0.7505
Demographic Parity Difference: 0.1283
Equal Opportunity Difference: 0.1978
Equalized Odds Difference: 0.1978
Unique values in prediction: [0 1]
Counts of unique values: [217 211]
------------------------------
Training with epsilon = 0.5
Accuracy: 0.7009
Precision: 0.3333
Recall: 0.2673
F1-score: 0.2967
Disparate Impact Ratio: 0.5605
Demographic Parity Difference: 0.0896
Equal Opportunity Difference: 0.3176
Equalized Odds Difference: 0.3176
Unique values in prediction: [0 1]
Counts of unique values: [347  81]
------------------------------
Training with epsilon = 1
Accuracy: 0.7079
Precision: 0.3605
Recall: 0.3069
F1-score: 0.3316
Disparate Impact Ratio: 0.2495
Demographic Parity Difference: 0.1719
Equal Opportunity Difference: 0.2904
Equalized Odds Difference: 0.2904
Unique values in prediction: [0 1]
Counts of unique values: [342  86]
-------------------------

In [ ]:
epsilon_values = [0.1, 0.5, 1, 5, 10]

for epsilon in epsilon_values:
    print(f"Training with epsilon = {epsilon}")
    dp_nb_model = DPNB(epsilon=epsilon, random_state=42)
    dp_nb_model.fit(X_train_balanced, y_train_balanced)

    y_pred_dp_nb = dp_nb_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_dp_nb)
    precision = precision_score(y_test, y_pred_dp_nb, average=indicator)
    recall = recall_score(y_test, y_pred_dp_nb, average=indicator)
    f1 = f1_score(y_test, y_pred_dp_nb, average=indicator)
    dir = calculate_disparate_impact_ratio(y_test, y_pred_dp_nb, sensitive_features)
    dpd = calculate_demographic_parity_difference(y_test, y_pred_dp_nb, sensitive_features)
    eopp = calculate_equal_opportunity_difference(y_test, y_pred_dp_nb, sensitive_features)
    eo = calculate_equalized_odds_difference(y_test, y_pred_dp_nb, sensitive_features)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Disparate Impact Ratio: {dir:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    print(f"Equal Opportunity Difference: {eopp:.4f}")
    print(f"Equalized Odds Difference: {eo:.4f}")


    # Count unique values in the prediction
    unique, counts = np.unique(y_pred_dp_nb, return_counts=True)
    print(f"Unique values in prediction: {unique}")
    print(f"Counts of unique values: {counts}")

    del dp_nb_model
    del y_pred_dp_nb

    diffpriv_nb_result[epsilon] = {
        'Model': ['Naive Bayes'],
        'Type': ['DiffPriv'],
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'DIR': dir,
        'DPD': dpd,
        'EOpp': eopp,
        'EO': eo,
    }

    print("-" * 30)

Training with epsilon = 0.1
Accuracy: 0.3271
Precision: 0.2274
Recall: 0.7723
F1-score: 0.3514
Disparate Impact Ratio: 0.6561
Demographic Parity Difference: 0.2920
Equal Opportunity Difference: 0.1750
Equalized Odds Difference: 0.3274
Unique values in prediction: [0 1]
Counts of unique values: [ 85 343]
------------------------------
Training with epsilon = 0.5
Accuracy: 0.7220
Precision: 0.3500
Recall: 0.2079
F1-score: 0.2609
Disparate Impact Ratio: 0.8710
Demographic Parity Difference: 0.0203
Equal Opportunity Difference: 0.2471
Equalized Odds Difference: 0.2471
Unique values in prediction: [0 1]
Counts of unique values: [368  60]
------------------------------
Training with epsilon = 1


/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be 

Accuracy: 0.4860
Precision: 0.2720
Recall: 0.7030
F1-score: 0.3923
Disparate Impact Ratio: 0.8051
Demographic Parity Difference: 0.1420
Equal Opportunity Difference: 0.2787
Equalized Odds Difference: 0.2787
Unique values in prediction: [0 1]
Counts of unique values: [167 261]
------------------------------
Training with epsilon = 5
Accuracy: 0.5491
Precision: 0.3244
Recall: 0.8416
F1-score: 0.4683
Disparate Impact Ratio: 0.7885
Demographic Parity Difference: 0.1341
Equal Opportunity Difference: 0.2574
Equalized Odds Difference: 0.2574
Unique values in prediction: [0 1]
Counts of unique values: [166 262]
------------------------------
Training with epsilon = 10
Accuracy: 0.7173
Precision: 0.4390
Recall: 0.7129
F1-score: 0.5434
Disparate Impact Ratio: 0.5529
Demographic Parity Difference: 0.1848
Equal Opportunity Difference: 0.1787
Equalized Odds Difference: 0.1817
Unique values in prediction: [0 1]
Counts of unique values: [264 164]
------------------------------


/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "


# Standard w/ fairlearn

In [ ]:
lr_model = LogisticRegression(random_state=42, max_iter=5000)
mitigator_lr = ExponentiatedGradient(lr_model, constraints=DemographicParity())

mitigator_lr.fit(X_train_balanced, y_train_balanced, sensitive_features=sensitive_features_train)
y_pred_lr_fair = mitigator_lr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_lr_fair)
precision = precision_score(y_test, y_pred_lr_fair, average=indicator)
recall = recall_score(y_test, y_pred_lr_fair, average=indicator)
f1 = f1_score(y_test, y_pred_lr_fair, average=indicator)

dir = calculate_disparate_impact_ratio(y_test, y_pred_lr_fair, sensitive_features)
dpd = calculate_demographic_parity_difference(y_test, y_pred_lr_fair, sensitive_features)
eopp = calculate_equal_opportunity_difference(y_test, y_pred_lr_fair, sensitive_features)
eo = calculate_equalized_odds_difference(y_test, y_pred_lr_fair, sensitive_features)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Disparate Impact Ratio: {dir:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equal Opportunity Difference: {eopp:.4f}")
print(f"Equalized Odds Difference: {eo:.4f}")

lr_fair_result = pd.DataFrame({
    'Model': ['Logistic Regression'],
    'Type': ['Standard w/ fairlearn'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-score': [f1],
    'DIR': [dir],
    'DPD': [dpd],
    'EOpp': [eopp],
    'EO': [eo]
})

Accuracy: 0.8551
Precision: 0.6667
Recall: 0.7723
F1-score: 0.7156
Disparate Impact Ratio: 0.8939
Demographic Parity Difference: 0.0318
Equal Opportunity Difference: 0.0265
Equalized Odds Difference: 0.0568


In [ ]:
rf_model = RandomForestClassifier(random_state=42)
mitigator_rf = ExponentiatedGradient(rf_model, constraints=DemographicParity())

mitigator_rf.fit(X_train_balanced, y_train_balanced, sensitive_features=sensitive_features_train)
y_pred_rf_fair = mitigator_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf_fair)
precision = precision_score(y_test, y_pred_rf_fair, average=indicator)
recall = recall_score(y_test, y_pred_rf_fair, average=indicator)
f1 = f1_score(y_test, y_pred_rf_fair, average=indicator)

dir = calculate_disparate_impact_ratio(y_test, y_pred_rf_fair, sensitive_features)
dpd = calculate_demographic_parity_difference(y_test, y_pred_rf_fair, sensitive_features)
eopp = calculate_equal_opportunity_difference(y_test, y_pred_rf_fair, sensitive_features)
eo = calculate_equalized_odds_difference(y_test, y_pred_rf_fair, sensitive_features)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Disparate Impact Ratio: {dir:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equal Opportunity Difference: {eopp:.4f}")
print(f"Equalized Odds Difference: {eo:.4f}")

rf_fair_result = pd.DataFrame({
    'Model': ['Random Forest'],
    'Type': ['Standard w/ fairlearn'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-score': [f1],
    'DIR': [dir],
    'DPD': [dpd],
    'EOpp': [eopp],
    'EO': [eo]
})

Accuracy: 0.8271
Precision: 0.5918
Recall: 0.8614
F1-score: 0.7016
Disparate Impact Ratio: 0.7027
Demographic Parity Difference: 0.1359
Equal Opportunity Difference: 0.1324
Equalized Odds Difference: 0.2239


In [ ]:
nb_model = GaussianNB()
mitigator_nb = ExponentiatedGradient(nb_model, constraints=DemographicParity())

mitigator_nb.fit(X_train_balanced, y_train_balanced, sensitive_features=sensitive_features_train)
y_pred_nb_fair = mitigator_nb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_nb_fair)
precision = precision_score(y_test, y_pred_nb_fair, average=indicator)
recall = recall_score(y_test, y_pred_nb_fair, average=indicator)
f1 = f1_score(y_test, y_pred_nb_fair, average=indicator)

dir = calculate_disparate_impact_ratio(y_test, y_pred_nb_fair, sensitive_features)
dpd = calculate_demographic_parity_difference(y_test, y_pred_nb_fair, sensitive_features)
eopp = calculate_equal_opportunity_difference(y_test, y_pred_nb_fair, sensitive_features)
eo = calculate_equalized_odds_difference(y_test, y_pred_nb_fair, sensitive_features)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Disparate Impact Ratio: {dir:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")
print(f"Equal Opportunity Difference: {eopp:.4f}")
print(f"Equalized Odds Difference: {eo:.4f}")

nb_fair_result = pd.DataFrame({
    'Model': ['Naive Bayes'],
    'Type': ['Standard w/ fairlearn'],
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1-score': [f1],
    'DIR': [dir],
    'DPD': [dpd],
    'EOpp': [eopp],
    'EO': [eo]
})

Accuracy: 0.7593
Precision: 0.4929
Recall: 0.6832
F1-score: 0.5726
Disparate Impact Ratio: 0.8573
Demographic Parity Difference: 0.0530
Equal Opportunity Difference: 0.0051
Equalized Odds Difference: 0.0726


# DiffPriv w/ fairlearn

In [ ]:
diffpriv_lr_fair_result = {}
diffpriv_rf_fair_result = {}
diffpriv_nb_fair_result = {}

In [ ]:
epsilon_values = [0.1, 0.5, 1, 5, 10]

for epsilon in epsilon_values:
    print(f"Training with epsilon = {epsilon}")
    dp_lr_model = DPLR(epsilon=epsilon, random_state=42, max_iter=5000)
    mitigator_lr = ExponentiatedGradient(dp_lr_model, constraints=DemographicParity())

    mitigator_lr.fit(X_train_balanced, y_train_balanced, sensitive_features=sensitive_features_train)

    y_pred_dp_lr = mitigator_lr.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_dp_lr)
    precision = precision_score(y_test, y_pred_dp_lr, average=indicator)
    recall = recall_score(y_test, y_pred_dp_lr, average=indicator)
    f1 = f1_score(y_test, y_pred_dp_lr, average=indicator)

    dir = calculate_disparate_impact_ratio(y_test, y_pred_dp_lr, sensitive_features)
    dpd = calculate_demographic_parity_difference(y_test, y_pred_dp_lr, sensitive_features)
    eopp = calculate_equal_opportunity_difference(y_test, y_pred_dp_lr, sensitive_features)
    eo = calculate_equalized_odds_difference(y_test, y_pred_dp_lr, sensitive_features)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    print(f"Disparate Impact Ratio: {dir:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    print(f"Equal Opportunity Difference: {eopp:.4f}")
    print(f"Equalized Odds Difference: {eo:.4f}")

    # Count unique values in the prediction
    unique, counts = np.unique(y_pred_dp_lr, return_counts=True)
    print(f"Unique values in prediction: {unique}")
    print(f"Counts of unique values: {counts}")

    del dp_lr_model
    del mitigator_lr

    diffpriv_lr_fair_result[epsilon] = {
        'Model': ['Logistic Regression'],
        'Type': ['DiffPriv w/ fairlearn'],
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'DIR': dir,
        'DPD': dpd,
        'EOpp': eopp,
        'EO': eo,
    }

    print("-" * 30)

Training with epsilon = 0.1


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter

Accuracy: 0.7477
Precision: 0.4386
Recall: 0.2475
F1-score: 0.3165
Disparate Impact Ratio: 0.4918
Demographic Parity Difference: 0.0738
Equal Opportunity Difference: 0.2941
Equalized Odds Difference: 0.2941
Unique values in prediction: [0 1]
Counts of unique values: [371  57]
------------------------------
Training with epsilon = 0.5


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter

Accuracy: 0.7477
Precision: 0.4386
Recall: 0.2475
F1-score: 0.3165
Disparate Impact Ratio: 0.4918
Demographic Parity Difference: 0.0738
Equal Opportunity Difference: 0.2199
Equalized Odds Difference: 0.2199
Unique values in prediction: [0 1]
Counts of unique values: [371  57]
------------------------------
Training with epsilon = 1


...

/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. 

Accuracy: 0.7407
Precision: 0.4219
Recall: 0.2673
F1-score: 0.3273
Disparate Impact Ratio: 0.4334
Demographic Parity Difference: 0.0934
Equal Opportunity Difference: 0.2434
Equalized Odds Difference: 0.2434
Unique values in prediction: [0 1]
Counts of unique values: [364  64]
------------------------------
Training with epsilon = 10


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/logistic_regression.py:231: PrivacyLeakWarning: Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.
  warnings.warn("Data norm has not been specified and will be calculated on the data provided.  This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter

Accuracy: 0.7640
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000
Disparate Impact Ratio: 1.0000
Demographic Parity Difference: 0.0000
Equal Opportunity Difference: 0.0000
Equalized Odds Difference: 0.0000
Unique values in prediction: [0]
Counts of unique values: [428]
------------------------------


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
epsilon_values = [0.1, 0.5, 1, 5, 10]

# Calculate bounds for each feature from the training data
min_bounds = X_train_balanced.min(axis=0).tolist()
max_bounds = X_train_balanced.max(axis=0).tolist()
bounds = (min_bounds, max_bounds)

# Get the unique classes from the training target variable
classes = np.unique(y_train_balanced)

for epsilon in epsilon_values:
    print(f"Training with epsilon = {epsilon}")
    # Pass the calculated bounds and classes to the DPRF constructor
    dp_rf_model = DPRF(epsilon=epsilon, random_state=42, bounds=bounds, classes=classes)
    mitigator_rf = ExponentiatedGradient(dp_rf_model, constraints=DemographicParity())

    mitigator_rf.fit(X_train_balanced, y_train_balanced, sensitive_features=sensitive_features_train)

    y_pred_dp_rf = mitigator_rf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_dp_rf)
    precision = precision_score(y_test, y_pred_dp_rf, average=indicator)
    recall = recall_score(y_test, y_pred_dp_rf, average=indicator)
    f1 = f1_score(y_test, y_pred_dp_rf, average=indicator)

    dir = calculate_disparate_impact_ratio(y_test, y_pred_dp_rf, sensitive_features)
    dpd = calculate_demographic_parity_difference(y_test, y_pred_dp_rf, sensitive_features)
    eopp = calculate_equal_opportunity_difference(y_test, y_pred_dp_rf, sensitive_features)
    eo = calculate_equalized_odds_difference(y_test, y_pred_dp_rf, sensitive_features)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    print(f"Disparate Impact Ratio: {dir:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    print(f"Equal Opportunity Difference: {eopp:.4f}")
    print(f"Equalized Odds Difference: {eo:.4f}")

    # Count unique values in the prediction
    unique, counts = np.unique(y_pred_dp_rf, return_counts=True)
    print(f"Unique values in prediction: {unique}")
    print(f"Counts of unique values: {counts}")

    del dp_rf_model
    del mitigator_rf

    diffpriv_rf_fair_result[epsilon] = {
        'Model': ['Random Forest'],
        'Type': ['DiffPriv w/ fairlearn'],
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'DIR': dir,
        'DPD': dpd,
        'EOpp': eopp,
        'EO': eo,
    }

    print("-" * 30)

/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "


Training with epsilon = 0.1


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/dif

Accuracy: 0.5327
Precision: 0.2838
Recall: 0.6436
F1-score: 0.3939
Disparate Impact Ratio: 0.9828
Demographic Parity Difference: 0.0093
Equal Opportunity Difference: 0.0963
Equalized Odds Difference: 0.0963
Unique values in prediction: [0 1]
Counts of unique values: [199 229]
------------------------------
Training with epsilon = 0.5


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/dif

...

/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/dif

Accuracy: 0.5911
Precision: 0.3150
Recall: 0.6238
F1-score: 0.4186
Disparate Impact Ratio: 0.8908
Demographic Parity Difference: 0.0562
Equal Opportunity Difference: 0.0728
Equalized Odds Difference: 0.0971
Unique values in prediction: [0 1]
Counts of unique values: [228 200]
------------------------------
Training with epsilon = 10


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/dif

Accuracy: 0.5888
Precision: 0.3272
Recall: 0.7030
F1-score: 0.4465
Disparate Impact Ratio: 0.9835
Demographic Parity Difference: 0.0084
Equal Opportunity Difference: 0.0926
Equalized Odds Difference: 0.0926
Unique values in prediction: [0 1]
Counts of unique values: [211 217]
------------------------------


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "


In [ ]:
epsilon_values = [0.1, 0.5, 1, 5, 10]

for epsilon in epsilon_values:
    print(f"Training with epsilon = {epsilon}")
    dp_nb_model = DPNB(epsilon=epsilon, random_state=42)
    mitigator_nb = ExponentiatedGradient(dp_nb_model, constraints=DemographicParity())

    mitigator_nb.fit(X_train_balanced, y_train_balanced, sensitive_features=sensitive_features_train)
    y_pred_dp_nb = mitigator_nb.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_dp_nb)
    precision = precision_score(y_test, y_pred_dp_nb, average=indicator)
    recall = recall_score(y_test, y_pred_dp_nb, average=indicator)
    f1 = f1_score(y_test, y_pred_dp_nb, average=indicator)

    dir = calculate_disparate_impact_ratio(y_test, y_pred_dp_nb, sensitive_features)
    dpd = calculate_demographic_parity_difference(y_test, y_pred_dp_nb, sensitive_features)
    eopp = calculate_equal_opportunity_difference(y_test, y_pred_dp_nb, sensitive_features)
    eo = calculate_equalized_odds_difference(y_test, y_pred_dp_nb, sensitive_features)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    print(f"Disparate Impact Ratio: {dir:.4f}")
    print(f"Demographic Parity Difference: {dpd:.4f}")
    print(f"Equal Opportunity Difference: {eopp:.4f}")
    print(f"Equalized Odds Difference: {eo:.4f}")

    unique, counts = np.unique(y_pred_dp_nb, return_counts=True)
    print(f"Unique values in prediction: {unique}")
    print(f"Counts of unique values: {counts}")

    del dp_nb_model
    del mitigator_nb

    diffpriv_nb_fair_result[epsilon] = {
        'Model': ['Naive Bayes'],
        'Type': ['DiffPriv w/ fairlearn'],
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1,
        'DIR': dir,
        'DPD': dpd,
        'EOpp': eopp,
        'EO': eo,
    }

    print("-" * 30)

Training with epsilon = 0.1


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not fu

Accuracy: 0.7173
Precision: 0.1875
Recall: 0.0594
F1-score: 0.0902
Disparate Impact Ratio: 0.9471
Demographic Parity Difference: 0.0040
Equal Opportunity Difference: 0.0779
Equalized Odds Difference: 0.0779
Unique values in prediction: [0 1]
Counts of unique values: [396  32]
------------------------------
Training with epsilon = 0.5


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not fu

Accuracy: 0.6682
Precision: 0.3119
Recall: 0.3366
F1-score: 0.3238
Disparate Impact Ratio: 0.9262
Demographic Parity Difference: 0.0200
Equal Opportunity Difference: 0.2515
Equalized Odds Difference: 0.2515
Unique values in prediction: [0 1]
Counts of unique values: [319 109]
------------------------------
Training with epsilon = 1


...

Accuracy: 0.2710
Precision: 0.2421
Recall: 0.9802
F1-score: 0.3882
Disparate Impact Ratio: 0.9981
Demographic Parity Difference: 0.0018
Equal Opportunity Difference: 0.0235
Equalized Odds Difference: 0.0235
Unique values in prediction: [0 1]
Counts of unique values: [ 19 409]
------------------------------
Training with epsilon = 5


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not fu

Accuracy: 0.5724
Precision: 0.3153
Recall: 0.6931
F1-score: 0.4334
Disparate Impact Ratio: 0.9175
Demographic Parity Difference: 0.0460
Equal Opportunity Difference: 0.1551
Equalized Odds Difference: 0.1551
Unique values in prediction: [0 1]
Counts of unique values: [206 222]
------------------------------
Training with epsilon = 10


/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
/usr/local/lib/python3.12/dist-packages/diffprivlib/models/naive_bayes.py:114: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "
/usr/local/lib/python3.12/dist-packages/diffprivlib/utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'sample_weight' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not fu

Accuracy: 0.6799
Precision: 0.4022
Recall: 0.7327
F1-score: 0.5193
Disparate Impact Ratio: 0.9288
Demographic Parity Difference: 0.0326
Equal Opportunity Difference: 0.1279
Equalized Odds Difference: 0.1279
Unique values in prediction: [0 1]
Counts of unique values: [244 184]
------------------------------


# Save Data

In [ ]:
# Combine the results for standard and standard w/ fairlearn classifiers
all_standard_results = pd.concat([lr_result, rf_result, nb_result,
                                 lr_fair_result, rf_fair_result, nb_fair_result])

# Round the numerical columns to four decimal places
numerical_cols = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'DIR', 'DPD', 'EOpp', 'EO']
all_standard_results[numerical_cols] = all_standard_results[numerical_cols].round(4)

# Save the combined results to a CSV file
all_standard_results.to_csv(f'{title}_standard_results.csv', index=False)

print(f"Standard and Standard w/ Fairlearn results saved to '{title}_standard_results.csv'")

Standard and Standard w/ Fairlearn results saved to 'aids_standard_results.csv'


In [ ]:
# Convert the dictionaries of results for DiffPriv and DiffPriv w/ fairlearn to DataFrames
diffpriv_lr_results_df = pd.DataFrame.from_dict(diffpriv_lr_result, orient='index')
diffpriv_rf_results_df = pd.DataFrame.from_dict(diffpriv_rf_result, orient='index')
diffpriv_nb_results_df = pd.DataFrame.from_dict(diffpriv_nb_result, orient='index')

# Add epsilon as a column
diffpriv_lr_results_df['epsilon'] = diffpriv_lr_results_df.index
diffpriv_rf_results_df['epsilon'] = diffpriv_rf_results_df.index
diffpriv_nb_results_df['epsilon'] = diffpriv_nb_results_df.index

# Combine the results for DiffPriv classifiers
all_diffpriv_results = pd.concat([diffpriv_lr_results_df, diffpriv_rf_results_df, diffpriv_nb_results_df])

# Flatten the list values in 'Model' and 'Type' columns
all_diffpriv_results['Model'] = all_diffpriv_results['Model'].apply(lambda x: x[0])
all_diffpriv_results['Type'] = all_diffpriv_results['Type'].apply(lambda x: x[0])


# Convert the dictionaries of results for DiffPriv w/ fairlearn to DataFrames
diffpriv_lr_fair_results_df = pd.DataFrame.from_dict(diffpriv_lr_fair_result, orient='index')
diffpriv_rf_fair_results_df = pd.DataFrame.from_dict(diffpriv_rf_fair_result, orient='index')
diffpriv_nb_fair_results_df = pd.DataFrame.from_dict(diffpriv_nb_fair_result, orient='index')

# Add epsilon as a column
diffpriv_lr_fair_results_df['epsilon'] = diffpriv_lr_fair_results_df.index
diffpriv_rf_fair_results_df['epsilon'] = diffpriv_rf_fair_results_df.index
diffpriv_nb_fair_results_df['epsilon'] = diffpriv_nb_fair_results_df.index


# Combine the results for DiffPriv w/ fairlearn classifiers
all_diffpriv_fair_results = pd.concat([diffpriv_lr_fair_results_df, diffpriv_rf_fair_results_df, diffpriv_nb_fair_results_df])

# Flatten the list values in 'Model' and 'Type' columns
all_diffpriv_fair_results['Model'] = all_diffpriv_fair_results['Model'].apply(lambda x: x[0])
all_diffpriv_fair_results['Type'] = all_diffpriv_fair_results['Type'].apply(lambda x: x[0])


# Combine all DiffPriv and DiffPriv w/ fairlearn results
all_diffpriv_combined_results = pd.concat([all_diffpriv_results, all_diffpriv_fair_results])

# Round the numerical columns to four decimal places
numerical_cols = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'DIR', 'DPD', 'EOpp', 'EO']
all_diffpriv_combined_results[numerical_cols] = all_diffpriv_combined_results[numerical_cols].round(4)

# Define the desired column order
desired_column_order = ['Model', 'Type', 'epsilon', 'Accuracy', 'Precision', 'Recall', 'F1-score', 'DIR', 'DPD', 'EOpp', 'EO']

# Reindex the DataFrame with the desired column order
all_diffpriv_combined_results = all_diffpriv_combined_results.reindex(columns=desired_column_order)

# Save the combined results to a CSV file
all_diffpriv_combined_results.to_csv(f'{title}_diffpriv_results.csv', index=False)

print(f"DiffPriv and DiffPriv w/ fairlearn results saved to '{title}_diffpriv_results.csv'")

DiffPriv and DiffPriv w/ fairlearn results saved to 'aids_diffpriv_results.csv'
